### Group Members:

- Hangchen Xie, 22-736-789
- Yanyang Gong, 23-744-063
- Yaojie Wang, 23-741-283

# Assignment 2: Gradient Descent

The goal of this exercise is to gain experience with a basic technique of Deep Learning, i.e., gradient descent.
A two-dimensional loss surface is created manually and gradient descent is implemented.
Several runs of gradient descent from different starting locations will be performed.
The loss surface and the detected minima are plotted together in one 3D plot.

## Compute the Gradient
The loss function is manually defined as $$\mathcal J_{\vec w}=40 \sin^3\left(\frac{1}{2}w_1+ w_2\right) + w_1^2 + w_2^2$$
The weights $\vec w = (w_1, w_2)^T$ shall be optimized such that the loss function has a minimum.

### Task 1: Compute the Gradient

The gradient $\nabla \mathcal J_{\vec w}$ is defined as the partial derivatives of the loss function with respect to the two variables $w_1$ and $w_2$.
We need to calculate it:

* $\frac{\partial \mathcal J}{\partial w_1} = 60 \sin^2\left(\frac{1}{2}w_1+ w_2\right)\cos\left(\frac{1}{2}w_1+ w_2\right)  + 2w_1$
* $\frac{\partial \mathcal J}{\partial w_2} = 120 \sin^2\left(\frac{1}{2}w_1+ w_2\right)\cos\left(\frac{1}{2}w_1+ w_2\right)   + 2w_2$

In [1]:
import sympy as sp

# define the symbols
w1, w2 = sp.symbols('w1 w2')

# define the loss function
J = 40 * sp.sin((w1/2) + w2)**3 + w1**2 + w2**2

# compute the gradients
grad_w1 = sp.diff(J, w1)
grad_w2 = sp.diff(J, w2)

print("Computed gradient w.r.t w1:", grad_w1)
print("Computed gradient w.r.t w2:", grad_w2)

Computed gradient w.r.t w1: 2*w1 + 60*sin(w1/2 + w2)**2*cos(w1/2 + w2)
Computed gradient w.r.t w2: 2*w2 + 120*sin(w1/2 + w2)**2*cos(w1/2 + w2)


### Task 2: Implement the Loss Function

Implement the loss function in Python, which takes a given $\vec w$ and returns $\mathcal J_{\vec w}$ according to the given loss function.

In [2]:
import numpy 

def loss(w):
    return 40 * numpy.sin((w[0]/2) + w[1])**3 + w[0]**2 + w[1]**2
  

### Task 3: Implement the Gradient

Implement the gradient as a function in Python, which takes a given $\vec w$ and returns $\nabla\mathcal J_{\vec w}$ according to the analytical result in Task 1.
Remember that the gradient needs to be computed and returned for both $w_1$ and $w_2$.

In [3]:
def gradient(w):
    grad_w1 = 60 * numpy.sin((w[0]/2) + w[1])**2 * numpy.cos((w[0]/2) + w[1]) + 2*w[0]
    grad_w2 = 120 * numpy.sin((w[0]/2) + w[1])**2 * numpy.cos((w[0]/2) + w[1]) + 2*w[1]
    return numpy.array([grad_w1, grad_w2])

### Test 1: Test Functions
The codes below call the loss function from Task 2 and the gradient function from Task 3 with $\vec w=(0,0)^T$ and then compare the return values with the given analytically computed values.
Please check your implementation if the tests cannot be passed.

Make sure your code can pass the test before moving to the next task.

In [4]:
w = numpy.zeros(2)

# analytically compute the expected values
expected_loss = 0.
expected_gradient = numpy.array((0.,0.))

# test loss function
assert abs(loss(w) - expected_loss) < 1e-8
assert numpy.all(numpy.abs(gradient(w) - expected_gradient) < 1e-8)
print("Tests passed")

Tests passed


## Implement Gradient Descent
The procedure of gradient decent is the repeated application of two steps:

* First, the gradient of the loss $\nabla\mathcal J_{\vec w}$ is computed based on the current value of the parameters $\vec w$.

* Second, the weights are updated by moving a small step in the direction of the negative gradient: $\vec w = \vec w - \eta\nabla\mathcal J_{\vec w}$

Optionally, the loss $\mathcal J_{\vec w}$ is computed to record the progress of the gradient descent.
Finally, one or more appropriate criteria need to be defined to decide when to stop the procedure.

### Task 4: Termination Criterion

(theoretical question) Define a proper termination criterion. Which error cases might occur and need to be considered?

Termination criterion: The termination criterion should be a threshold for the norm of the gradient. If the norm of the gradient is smaller than the threshold and little change in the loss after fixed number of iterations, the algorithm should stop.

Error Cases:
1. Reaches local minimum: The algorithm may stop at a local minimum instead of the global minimum.
2. Gets suck in plateau: The algorithm may stop at a plateau where the gradient is very small.
3. Oscillates / diverges: The algorithm may oscillate around the minimum or diverge if the learning rate is too large.
4. Jumps over the minimum: The algorithm may jump over the minimum if the learning rate is too large.

### Task 5: Implement Gradient Descent

Implement a function that performs the gradient descent. This function should take as parameters an initial weight vector $\vec w$ and a learning rate $\eta$, and make use of the gradient function implemented in Task 3 and, possibly, the loss function from Task 2.
It should return the optimized weight vector $\vec w^*$. Incorporate the termination criterion designed in Task 4.

In [5]:
def gradient_descent(w, eta=0.01):
  # copy the weights to not modify the original values
  w_star = w.copy()

  # perform iterative gradient descent
  while True:
    # compute the gradient
    grad = gradient(w_star)

    # update the weights
    w_star -=  eta * grad

    # include additional termination criteria?
    # if loss(w_star) < 0:
    #    w_star  -= 1.1 * eta * grad
    # else:
    #    w_star  -= 0.5 * eta * grad
    if loss(w_star) < 1e-8:
      break
      

  return w_star

## Evaluate Gradient Descent

### Task 6: Evaluate Gradient Descent
Call the gradient descent function from Task 5 1000 times with different random initialized weights $\vec w\in[-10,10]^2$ and a learning rate of $\eta=0.01$. Store the resulting optimized weight vectors in a list.

In [6]:
stored_weights = []

for i in range(1000):
  # create random weight vector
  w = numpy.random.uniform(-10, 10, 2)
  # call gradient descent
  w_star = gradient_descent(w, 0.01)
  # store it in the list
  stored_weights.append(w_star)

KeyboardInterrupt: 

### Test 2: Check Minima

Counting the number of local minima in our loss function, we reach a total of 11. Please use this function to verify that your implementation could reach this number at maximum.

Again, make sure you pass the test before moving to the next task.

In [ ]:
maximum_number_of_minima = 11

# compute the number of reached minima
minima = []
for w_star in stored_weights:
  # check if this weight vector is far enough
  # from all previously stored vectors
  if all(numpy.linalg.norm(w_star-w) > 1e-3 for w in minima):
    minima.append(w_star)
number_of_minima = len(minima)

assert number_of_minima <= maximum_number_of_minima

print("Check passed. The number of minima", number_of_minima, "is lower than or equal to the maximum", maximum_number_of_minima)

### Task 7: Find the Global Minimum

Find the global minimum of our error function by evaluating the obtained optimized weight vectors from Task 6.
Print the minimum and its loss value.

In [ ]:
# find the lowest loss
minimum_loss = numpy.inf
minimum_weights = None

for w_star in stored_weights:
  J = loss(w_star)
  if J < minimum_loss:
    minimum_loss = J
    minimum_weights = w_star

print("The minimum loss value of:", minimum_loss, "was found for minimum", minimum_weights)

## Plot Error Surface and Points

### Task 8: Loss Surface Plot

Plot the error surface of the given loss function. Limit range $\vec w\in[-20,20]^2$. For each of the optimized weights from Task 6, plot a marker into the 3D plot. An example can be found in the slides.

When plotting the resulting optimized weights $\vec w=(w_1, w_2)^T$, we need to define the third coordinate. What should this coordinate be?


In [ ]:
from matplotlib import pyplot

# create 3D axis
figure = pyplot.figure()
axis = figure.add_subplot(111, projection='3d', azim = -40, elev=50)

# define range to plot
w_range = numpy.linspace(-20, 20, 100)
w1, w2 = numpy.meshgrid(w_range, w_range)

# compute loss for w1 and w2
J = loss([w1, w2])

# plot surface with jet colormap
axis.plot_surface(w1, w2, J, cmap="jet", alpha=0.7)

# plot resulting points in 3D
for w_star in stored_weights:
  # compute the z-position
  z = loss(w_star)
  # plot as 3D point
  axis.plot([w_star[0]], [w_star[1]], [z], "kx")